In [ ]:
# windows 也有这种坑爹的时候, 键盘的 shift 失灵. 可以打开屏幕键盘后解决

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, GenerationConfig
from transformers import BitsAndBytesConfig

c:\tech\Anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
quantization_config = BitsAndBytesConfig(
    False,
    True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model_dir = r"G:\code\pretrain_model_dir\_modelscope\baichuan-inc\Baichuan2-13B-Chat"
tokenizer = AutoTokenizer.from_pretrained(
    model_dir, device_map="auto", trust_remote_code=True, torch_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
)
model.generation_config = GenerationConfig.from_pretrained(model_dir)

print(model.device, model.dtype)

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


bin c:\tech\Anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


Loading checkpoint shards: 100%|██████████| 3/3 [00:47<00:00, 15.79s/it]

cuda:0 torch.float16


In [163]:
tokenizer.convert_ids_to_tokens([  195, 92955, 92342, 92955, 83546, 92333,  2081,   196])

['<reserved_106>', '讲', '一', '讲', '牛顿', '的', '发现', '<reserved_107>']

In [3]:
tokenizer.chat_template = """
{% for message in messages %}
{% if message['role'] == 'user' %}
{{ '<reserved_106>' + message['content'].strip() }}
{% elif message['role'] == 'system' %}
{{ message['content'].strip() }}
{% elif message['role'] == 'assistant' %}
{{ '<reserved_107>'  + message['content'] }}
{% endif %}
{% if loop.last and message['role'] != 'assistant' %}
{{ '<reserved_107>' }}
{% endif %}
{% endfor %}
""".strip().replace("\n", "")

In [158]:
# tokenizer.chat_template = """
# {% set ns = namespace(text='') %}
# {% for message in messages %}
# {% if message['role'] == 'user' %}
# {% set ns.text = ns.text + '<reserved_106>' + message['content'].strip() %}
# {% elif message['role'] == 'system' %}
# {% set ns.text = ns.text + message['content'].strip() %}
# {% elif message['role'] == 'assistant' %}
# {% set ns.text = ns.text + '<reserved_107>'  + message['content'] %}
# {% endif %}
# {% if loop.last and message['role'] != 'assistant' %}
# {% set ns.text = ns.text + '<reserved_107>' %}
# {% endif %}
# {% endfor %}

# {% if ns.text|length > 10 %} {# 如果 text 的长度超过 n #}
#   {{ ns.text[-10:] }} {# 那么输出 text 的最后 n 个字符 #}
# {% else %} {# 如果 text 的长度不超过 n #}
#   {{ ns.text }} {# 那么输出 text 的全部内容 #}
# {% endif %}
# """.strip().replace("\n", "")

In [4]:
tokenizer.chat_template

"{% for message in messages %}{% if message['role'] == 'user' %}{{ '<reserved_106>' + message['content'].strip() }}{% elif message['role'] == 'system' %}{{ message['content'].strip() }}{% elif message['role'] == 'assistant' %}{{ '<reserved_107>'  + message['content'] }}{% endif %}{% if loop.last and message['role'] != 'assistant' %}{{ '<reserved_107>' }}{% endif %}{% endfor %}"

# 感觉也不是很完善, 我还不知道怎么截断回答. jinja 本身模板应该是做不到这一点的, 因为还没有变成 token, 只是文本很难判断具体长度

应该从 `apply_chat_template` 方法入手

In [5]:
chat = [
    {"role": "user", "content": "讲一讲牛顿的发现"},
    {"role": "user", "content": "讲一讲牛顿的发现"},
]

tokenizer.apply_chat_template(chat, tokenize=False)

'<reserved_106>讲一讲牛顿的发现<reserved_106>讲一讲牛顿的发现<reserved_107>'

In [6]:
chat = [
    {"role": "assistant", "content": "讲一讲牛顿的发现"},
]

tokenizer.apply_chat_template(chat, tokenize=False)

'<reserved_107>讲一讲牛顿的发现'

In [7]:
pred = model.generate(**tokenizer("讲一讲牛顿的发现", return_tensors="pt").to('cuda:0'), max_new_tokens=512)
print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))

讲一讲牛顿的发现



In [24]:
from transformers import pipeline, Conversation, ConversationalPipeline

# 这个长度不设置有点坑, 默认怎么会是 20, 都不知道是哪里设置的
model.config.max_length = 4096
tokenizer.pad_token_id = tokenizer.eos_token_id
chatbot = pipeline("conversational", model=model, tokenizer=tokenizer)
conversation = Conversation("讲一讲牛顿的发现")
conversation = chatbot(conversation)
print(conversation.generated_responses[-1])

Both `max_new_tokens` (=2048) and `max_length`(=4096) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


艾萨克·牛顿（Isaac Newton）是英国的一位杰出的科学家、数学家和哲学家，他生活在17世纪末和18世纪初。他的发现和理论对科学革命产生了深远的影响，被誉为科学史上最伟大的科学家之一。

牛顿的主要发现包括三个方面：运动定律、万有引力定律和光学。

1. 运动定律：牛顿提出了三大运动定律，即惯性定律、力与加速度定律以及作用与反作用定律。这些定律构成了经典力学的基础，描述了物体在受力时的运动状态。其中，惯性定律指出，物体在没有受到外力作用时，保持静止或匀速直线运动；力与加速度定律表明，物体所受的力与其加速度成正比，且与物体的质量成反比；作用与反作用定律则指出，作用力和反作用力大小相等，方向相反。

2. 万有引力定律：牛顿通过研究天体运动发现了万有引力定律。这个定律表明，两个物体之间的引力与它们的质量之积成正比，与它们之间的距离成反比。这一发现解释了地球上的重力现象以及天体间的运动规律。万有引力定律的发现使得人们能够预测和计算天体的运动，为后来的航天技术和地球科学的发展奠定了基础。

3. 光学：牛顿在光学领域的贡献主要体现在他发现了白光是由多种颜色组成的，这一发现被称为光的色散实验。此外，他还发明了反射式望远镜，通过使用曲面镜子将光线反向反射到观察者眼中，从而减少了球面像差和其他光学误差，提高了观测效果。

总之，牛顿的发现和理论对科学界产生了深远的影响，为现代科学技术的发展奠定了基础。他的成就不仅体现在物理学领域，还涉及到数学、天文学和哲学等多个方面，展示了他在科学领域的全面才能。


In [15]:
conversation.add_message({"role": "user", "content": "我刚刚提到了谁, 还有谁和他一样伟大, 举出三个例子"})
conversation = chatbot(conversation)
print(conversation.generated_responses[-1])

Both `max_new_tokens` (=2048) and `max_length`(=4096) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


你提到的应该是艾萨克·牛顿，与他同样伟大的科学家有很多，以下是其中的三位：

1. 阿尔伯特·爱因斯坦：他是20世纪最伟大的物理学家之一，提出了相对论，包括狭义相对论和广义相对论。他的质能方程E=mc²揭示了物质和能量之间的关系，对核能和原子弹的发展产生了重大影响。

2. 尼古拉·特斯拉：他是19世纪的塞尔维亚裔美国发明家、电气工程师和物理学家，对电力系统和交流电的发展做出了重要贡献。他的许多发明和创新，如特斯拉线圈和特斯拉发动机，至今仍被广泛应用于各种领域。

3. 查尔斯·达尔文：他是英国的自然学家和生物学家，他的物种起源理论彻底改变了我们对生物进化和人类起源的理解。他的理论被认为是现代生物学和遗传学的基石。


In [49]:
conversation.add_message({"role": "user", "content": "矩阵乘法是什么?"})
conversation = Conversation("矩阵乘法是什么?")
conversation = chatbot(conversation)
print(conversation.generated_responses[-1])

Both `max_new_tokens` (=2048) and `max_length`(=4096) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


矩阵乘法是一种数学运算，用于计算两个矩阵的乘积。给定两个矩阵A和B，它们的元素都是按照行和列索引的数值。矩阵乘法的结果是一个新的矩阵C，其元素由将A的对应元素与B的对应元素的乘积之和给出。

矩阵乘法具有以下性质：
1. 结合律：(A * B) * C = A * (B * C)
2. 分配律：A * (B + C) = A * B + A * C
3. 存在单位矩阵I，使得AI = IA = I，其中I是单位矩阵（对角线上的所有元素为1，其余元素为0）。
4. 存在逆矩阵，对于方阵A，如果A * A^(-1) = A^(-1) * A = I，则称A可逆，且A^(-1)是其逆矩阵。

矩阵乘法在计算机科学、线性代数、物理学等领域有广泛应用。例如，在机器学习和神经网络中，矩阵乘法用于计算数据点和模型参数之间的关系。


# 试试怎么和 langchain 的 chatmodel 结合

In [18]:
from langchain.llms.huggingface_hub import HuggingFaceHub  # 这个是用于在线模型的
from langchain.llms.huggingface_pipeline import HuggingFacePipeline  # 这个是用于本地模型
from langchain.chat_models.base import SimpleChatModel

# pipe = pipeline(
#     "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, repetition_penalty=1.1
# )
# 不能直接用, 它只接受 conversation 实例, 不支持 string
# llm = HuggingFacePipeline(pipeline=chatbot)

In [40]:
from typing import Any, List, Optional


from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.schema.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage


class BaichuanChatModel(SimpleChatModel):
    """
    实现 baichaun 的 chat model
    """
    model: Any = None
    # model_pipeline: ConversationalPipeline = None
    tokenizer: Any = None
    user_token: str = "<reserved_106>"
    ai_token: str = "<reserved_107>"

    @property
    def _llm_type(self) -> str:
        """Return type of chat model."""
        return "baichuan-chat"

    def _call(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        # TODO: 先忽略 run_manager
        # TODO: 先忽略 stop
        assert self.model is not None, "model is not initialized, 需要传入一个 baichuan chat 模型"
        # assert self.model_pipeline is not None, "model_pipeline is not initialized, 需要传入一个 baichuan chat pipeline"
        assert self.tokenizer is not None, "tokenizer is not initialized, 需要传入一个 baichuan chat tokenizer"
        text = ""
        for index, message in enumerate(messages):
            if message.type == "system":
                if index != 0:
                    raise Exception("system message must be the first message")
                text += message.content
            elif message.type == "human":
                text += self.user_token + message.content
            elif message.type == "ai":
                text += self.ai_token + message.content
            else:
                raise NotImplementedError(f"message type {messages.type} is not supported")
        
        if messages[-1].type != "ai":
            text += self.ai_token
        
        input_ids = tokenizer.encode(text)
        input_ids = torch.LongTensor([input_ids]).to(self.model.device)

        pred = self.model.generate(input_ids, generation_config=self.model.generation_config)
        response = tokenizer.decode(pred.cpu()[0][len(input_ids[0]):], skip_special_tokens=True)

        return response

In [42]:
llm = BaichuanChatModel(model=model, tokenizer=tokenizer)

In [43]:
messages = [
    SystemMessage(content="You're a helpful assistant"), 
    HumanMessage(content="What is the purpose of model regularization?")
]
llm.invoke(messages)

AIMessage(content='Model regularization refers to adding additional constraints or restrictions on the model, so that the model can better fit the training data and avoid overfitting.The purpose of model regularization includes: 1. Reduce the complexity of the model, thereby reducing the risk of overfitting.\n2. By adding constraints on the model parameters, it can be more accurate in predicting new data.\n3. It can also improve the performance of the model by preventing the model from being too sensitive to noise or small changes in input data.')

In [47]:
messages = [
    # SystemMessage(content="你是一个很乐于助人的伙伴, 会为大家详细的解答问题, 不只是简单的回答, 而是会详细的解释, 规划每一个解题步骤"), 
    HumanMessage(content="矩阵乘法是什么?")
]
print(llm.invoke(messages).content)

矩阵乘法是一种数学运算，用于计算两个矩阵的乘积。给定两个矩阵A和B，它们的元素都是按行排列的，我们可以通过以下方式计算它们的乘积：

C(i, j) = A(i, 1) * B(1, j) + A(i, 2) * B(2, j) + ... + A(i, n) * B(n, j)

其中，C是矩阵A和B的乘积，n是矩阵B的列数，i是矩阵A的行数，j是矩阵C的列数。

矩阵乘法具有以下性质：

1. 结合律：(A * B) * C = A * (B * C)
2. 分配律：A * (B + C) = A * B + A * C
3. 交换律：A * B = B * A（当且仅当A和B的维数相同时成立）
4. 单位矩阵I与任意矩阵A相乘得到矩阵A，即I * A = A。

矩阵乘法在许多领域都有广泛的应用，如线性代数、微积分、图像处理、数据挖掘等。
